## 1.1 행렬의 곱셈을 이용한 순전파(Forward Propagation)

![Alt text]( neuralnetwork_final.png )

- 위와 같은 인공 신경망을 케라스로 구현해 본다면 아래와 같이 짧은 코드로 구현할 수 있다.

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
model = Sequential() # 층을 추가할 준비
model.add( Dense( 8, input_dim = 4, activation = 'relu' ) )
# 입력층( 4 )과 다음 은닉층( 8 ) 그리고 은닉층의 활성화 함수는 relu
model.add( Dense( 8, activation = 'relu' ) )
# 은닉층(8)의 활성화 함숮는 relu
model.add( Dense( 3, activation = 'softmax' ) )
# 은닉층( 8 )의 활성화 함수는 softmax

- 인공 신경망에서 입력층에서 출력층 방향으로 연산을 진행하는 과정을 순전파(Forward Propagation)라고 한다.
- 다르게 말하면 주어진 입력으로부터 예측값을 계산하는 과정을 순전파라고 한다.
- 벡터와 행렬 연산이 순전파 과정에서 층(layer)마다 적용이 된다.
- 케라스로 인공 신경망을 만들면 이러한 연산 과정을 자세히 이해하지 않아도 모델을 만들 수는 있지만, Numpy 등을 통해 인공 신경망을 로우 레벨로 개발하고 있다면 인공 신경망 내부 연산에 사용하는 행렬 크기를 고려해야 인공 신경망을 구현할 수 있다.

### 1) layer 1의 행렬 크기 추정하기

- 각 층을 기준으로 입력과 출력의 개수 정리
    입력층 : 4개의 입력과 8개의 출력
    은닉층1 : 8개의 입력과 8개의 출력
    은닉층2 : 8개의 입력과 3개의 출력
    출력층 : 3개의 입력과 3개의 출력
    
- 입력층을 layer 0, 은닉층 1을 layer 1, 은닉층 2를 layer 2, 출력층을 layer 3으로 정할 때

- 각 layer 마다 가중치와 편향 행렬의 크기를 추정
- 가중치 행렬에 입력 행렬을 곱하는 경우와 입력 행렬에 가중치 행렬을 곱하는 경우가 있으나 후자를 기준으로 정리
- 배치 크기는 1로 한다. 이 경우 layer 1에서 처음 입력으로 들어오는 입력 행렬 x의 크기는 1 x 4로 행벡터에 해당한다.( 만약 미니 배치 학습을 가정할 경우, x의 크기는 '배치 크기 x 4'가 된다.)

- 입력 행렬, 가중치 행렬, 편향 행렬 출력 행렬은 다음과 같은 크기 관계를 가진다.

\begin{equation*}
    X_{n\ \text{x}\ n} * W_{n\ \text{x}\ j} + B_{m\ \text{x}\ j} = Y_{n\text{x}\ j}
\end{equation*}

-> W와 B는 뉴런 사이에 선과 같은 것

- layer 1의 입력 행렬 x의 크기는 1 x 4이고, layer 1의 출력은 8개이므로, 그에 따른 출력 행렬의 Y의 크기는 1 x 8이 된다.

\begin{equation*}
    X_{1\ \text{x}\ 4} * W_{n\ \text{x}\ j} + B_{m\ \text{x}\ j} = Y_{1\text{x}\ 8}
\end{equation*}

- 가중치 행렬 W의 행은 입력 행렬 X의 열과 같아야 하므로 아래와 같다.

\begin{equation*}
    X_{1\ \text{x}\ 4} * W_{4\ \text{x}\ j} + B_{m\ \text{x}\ j} = Y_{1\text{x}\ 8}
\end{equation*}

- 편향 행렬 B는 출력 행렬 Y의 크기에 영향을 주지 않으므로 편향 행렬 B의 크기는 출력 행렬 Y의 크기와 같다.

\begin{equation*}
    X_{1\ \text{x}\ 4} * W_{4\ \text{x}\ j} + B_{1\ \text{x}\ 8} = Y_{1\text{x}\ 8}
\end{equation*}

- 가중치 행렬 W의 열은 출력 행렬 Y의 열과 동일해야 한다.

\begin{equation*}
    X_{1\ \text{x}\ 4} * W_{4\ \text{x}\ 8} + B_{1\ \text{x}\ 8} = Y_{1\text{x}\ 8}
\end{equation*}

- layer 1의 가중치 행렬과 편향 행렬의 크기를 구했다.
- layer 1의 출력 행렬 Y는 layer 2에서는 입력 행렬 X가 된다.

### 2) layer 2와 layer 3의 행렬 크기 추정

- layer 1의 행렬 크기 추정 방식을 반복하여 layer 2와 layer 3에서의 가중치 행렬과 편향 행렬의 크기를 구할 수 있다.
- 비록 은닉층과 출력층에 황성화 함수가 존재하지만 활성화 함수는 행렬의 크기에 영향을 주지 않는다.

\begin{equation*}
    layer 2 : X_{1\ \text{x}\ 8} x W_{8\ \text{x}\ 8} + B_{1\ \text{x}\ 8} = Y_{1\ \text{x}\ 8}
\end{equation*}

\begin{equation*}
    layer 3 : X_{1\ \text{x}\ 8} x W_{8\ \text{x}\ 3} + B_{1\ \text{x}\ 3} = Y_{1\ \text{x}\ 3}
\end{equation*}

- 숫자를 비교하여 같아야 하는 값을 파악 입력값, 결과값 등

- 인공 신경망이 입력층에서 은닉층을 지나 출력층에서 예측값을 계산하기까지의 과정을 행렬 연산으로 가정하여 행렬의 크기를 추정해 보았다.
- 순전파를 진행하고 예측값을 구하고나서, 인공 신경망이 해야할 일은 예측값과 실제값으로부터 오차를 계산하고, 오차로부터 가중치와 편향을 업데이트하는 일이다. 이를 인공 신경망의 학습 단계라 한다.
- 인공 신경망은 순전파와는 반대 방향으로 연산을 진행하며 가중치와 편향을 업데이트하는데, 이 과정을 역전파( Back Propagation )라 한다.

## 1.2 순전파

![Alt text]( forward_propagation.png )

- 활성화 함수, 은닉층의 수, 각 은닉층의 뉴런 수 등 딥 러닝 모델을 설계하고나면 입력값은 입력층, 은닉층을 지나면서 각 층에서의 가중치와 함께 연산되며 출력층으로 향한다. 그리고 출력층에서 모든 연산을 마친 예측값이 나오게 된다.
- 입력층에서 출력층 방향으로 예측값의 연산이 진행되는 과정을 순전파라 한다.

## 1.3 손실 함수

![Alt text]( forward_loss_function.png )

- 손실 함수는 실제값의 예측값의 차이를 수치화해주는 함수이다.
- 두 값의 차이,  즉, 오차가 클 수록 손실 함수의 값은 크고 오차가 작을 수록 손실 함수의 값은 작아진다.
- 희귀에서는 평균 제곱 오차, 분류 문제에서는 크로스 엔트로피를 손실 함수로 사용한다.
- 손실 함수의 값을 최소화하는 두개의 매개 변수인 가중치 W와 편향 b를 찾아가는 것이 딥 러닝 학습 과정이므로 손실 함수 선정은 매우 중요하다.

### 1) MSE( Mean Squared Error, MSE )

- 오차 제곱 평균( 평균 제곱 오차 )를 의미, 연속형 변수를 예측할 때 사용

\begin{equation*}
    -\frac{1}{N} \sum (y - \hat{y})^2
\{\end{equation*}

### 2) 크로스 엔트로피( Cross-Entropy )

- y : 실제값( 0 or 1 )
- \begin(equation*)\hat{y} : 예측값( 확률 )\end(equation*)

\begin{equation*}
    -\sum ylog\hat{y}
\end{equation*}

- 낮은 확률로 예측해서 맞추거나, 높은 확률로 예측해서 틀리는 경우 loss가 더 크다.
- 이진 분류( binary classfication )의 경우 binary_crossentropy를 사용하고 다중 클래스 분류( multi-class classification )일 경우 categorical_crossentropy를 사용

## 1.4 옵티마이저( Optimizer )

![Alt text]( optimizer.png )

- 손실 함수의 값을 줄여나가면서 학습하는 방법은 어떤 옵티마이저를 사용하느냐에 따라 달라진다.
- 옵티마이저를 이해하기 전에 배치( batch )라는 개념을 이해해야 한다.
    - 배치는 가중치 등의 매개 변수의 값을 조정하기 위해 사용하는 데이터의 양을 말한다.
    - 전체 데이터를 가지고 매개 변수의 값을 조정할 수도 있고, 정해준 양의 데이터만 가지고도 매개 변수의 값을 조정할 수도 있다.
    

### 1) 배치 경사 하강법( Batch Gradient Descent )

- 가장 기본적인 경사 하강법이다.
- 배치 경사 하강법은 옵티마이저 중 하나로 오차( loss )를 구할 때 전체 데이터를 고려한다.
- 머신 러닝에서는 1번의 훈련 횟수를 1 에포크라고 하는데, 배치 경사 하강법은 한 번의 에포크에 모든 매개 변수 업데이트를 단 한 번 수행한다. 배치 경사 하강법은 전체 데이터를 고려해서 학습하므로 에포크당 시간이 오래 걸리며, 메모리를 크게 요구한다는 단점이 있으나 글로벌 미니멈을 찾을 수 있다는 장점이 있다.

keras Code : model.fit( X_train, y_train, batch_size = len( X_train ))

### 2) 확률적 경사 하강법 ( Stochastic Gradient Descent, SGD )

- 기존의 배치 경사 하강법은 전체 데이터에 대해서 계싼을 하다보니 시간이 너무 오래 걸린다는 단점이 있다.
- 확률적 경사 하강법은 매개변수 값을 조정시 전체 데이터가 아니라 랜덤으로 선택한 하나의 데이터에 대해서만 계산하는 방법이다.
- 더 적은 데이터를 사용하므로 더 빠르게 계산할 수 있다.

![Alt text]( SGD.png )

- 매개 변수의 변경폭이 불안정하고, 때로는 배치 경사 하강법보다 정확도가 낮을 수도 있지만 속도만큼은 배치 경사 하강법보다 빠르다는 장점이 있다.

keras Code : model.fit( X_train, y_train, batch_size = 1 )

### 3) 미니 배치 경사 하강법( Mini-Batch Gradient Descent )

- 전체 데이터도 아니고, 1개의 데이터도 아니고 정해진 양에 대해서만 계산하여 매개 변수의 값을 조정하는 경사 하강법을 미니 배치 경사 하강법이라 한다. 
- 미니 배치 경사 하강법은 전체 데이터를 계싼하는 것보다 빠르며, SGD 보다 안정적이라는 장점이 있다.
- 실제 많이 사용되는 경사 하강법이다.

keras Code : model.fit( X_train, y_train, batch_size = 32 ) # 배치크기 32

### 4) 모멘텀( Momentum )

- 모멘텀은 관성이라는 물리학의 법칙을 응용한 방법이다.
- 모멘텀 SGD는 경사 하강법에 관성을 더 해준다.
- 모멘텀은 SGD에서 계산된 접선의 기울기에 한 시점( step ) 전의 접선의 기울기값을 일정한 비율마큼 반영한다.
- 이렇게 하면 마치 언덕에서 공이 내려올 때, 중간에 작은 웅덩이에 빠지더라도 관성의 힘으로 넘어서는 효과를 줄 수 있다.

![Alt text]( localminimum.png )

- 로컬 미니멈에 도달하였을 때, 기울기가 0이라서 기존의 경사 하강법이라면 이를 글로벌 미니멈으로 잘 못 인식하여 계산이 끝났을 상황이라도 모멘텀은 관성의 힘을 빌리면 값이 조절되면서 로컬 미니멈에서 탈출하는 효과를 얻을 수 있다.

Keras Code : keras.optimizers.SGD( lr = 0.01, momentum = 0.9 )

### 5) 아다그라드( Adagrad )

- 매개 변수들은 각자 의미하는 바가 다른데, 모든 매개 변수에 돌이한 합습률( learning rate )를 적용하는 것은 비효율적이다.
- 아다그리다드는 각 매개 변수에 서로 다른 학습률을 적용시킨다.
- 이 때 변화가 많은 매개 변수는 학습률이 작게 설정되고 변화가 적은 매개 변수는 학습률을 높게 설정시킨다.

Keras Code : keras.optimizers.Adagrad( lr = 0.01, epsilon = 1e-6 )

### 6) 알엠에스프롭( RMSprop )

- 아다그라드는 학습을 계속 진행한 경우에는, 나중에 가서는 학습률이 지나치게 떨어진다는 단점이 있는데 다른 수식으로 대체하여 이러한 단점을 개선하였다.

Keras Code : keras.optimizers.RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-6 )

### 7) 아담( Adam )

- 아담은 알엠네스프롭과 모멘텀 두 가지를 합친 듯한 방법으로, 방향과 학습률 두 가지를 모두 잡기 위한 방법이다.

Keras Code : keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False )

- https://keras.io/api/optimizers/ => 옵티마이저스에 대한 자료

# 2. 딥러닝 학습방법 - 역전파( Back Propagation )

- 인공 신경망이 순전파 과정을 진행하여 예측값과 실제값과 실제값의 오차를 계산하였을 때 어떻게 역전파 과정에서 경사 하강법을 사용하여 가중치를 업데이트하는 계략적인 과정 이해

## 1. 인공 신경망 이해( Neura Network Overview )

- 입력층, 은닉층, 출력층의 3개의 층을 갖은 인공 신경망
- 입력층은 두 개의 입력과 두 개의 은닉층 뉴런, 두 개의 출력층 뉴런 사용
- 은닉층과 출력층의 모든 뉴런은 활성화 함수로 시그모이드 함수 사용 전제

![Alt text]( nn1_final.png )

- 위 인공 신경망에서 은닉층과 출력층의 모든 뉴런에 변수 z가 있는데 변수 z는 이전층의 모든 입력이 각각의 가중치와 곱해진 값을 모두 더해진 가중합을 의미한다,
- 변수 z의 값은 아직 시그모이드 함수를 거치지 않은 상태이다. 즉, 확성화 함수의 입력을 의미한다.
- 변수 z 우측의 변수 h와 o는 각 뉴런의 출력을 의미한다.

## 1. 순전파 ( Forward Propagation )

![Alt text]( nn2_final_final.png )


- 각 입력은 입력층에서 은닉층 방향으로 향하면서 각 입력에 해당하는 가중치와 곱해지고, 결과적으로 가중합으로 계산되어 은닉층 뉴런의 시그모이드 함수의 입력값이 된다.
- z1과 z2는 시그모이드 함수의 입력으로 사용되는 각각의 값에 해당한다.


z1 = W1x1 + W2x2 = 0.3 x 0.1 + 0.25 x 0.2 = 0.08   
z2 = W3x1 + W4x2 = 0.4 x 0.1 + 0.35 x 0.2 = 0.11

- z1과 z2는 각각의 은닉층 뉴런에서 시그모이드 함수를 지나게 되는데 시그모이드 함수가 리턴하는 은닉층 뉴런의 최종 출력값 h1과 h2는 다음과 같다. 

h1 = sigmoid( z1 ) = 0.51998934   
h2 = sigmoid( z2 ) = 0.52747230

- h1과 h2값은 다시 출력층의 뉴런으로 전달되어 각각의 값에 해당되는 가중치와 곱해지고, 다시 가중합이 되어 출력층 뉴런의 시그모이드 입력값이 된다. z3와 z4는 다음과 같다.

z3 = W5xh1 + W6xh2 = 0.45 x h1 + 0.4 x h2 = 0.44498412   
z4 = W7xh1 + W8xh2 = 0.7 x h1 + 0.6 x h2 = 0.68047592

- z3과 z4가 출력층 뉴런에서 시그모이드 함수를 지난 값은 인공신경망의 최종적으로 계산된 출력값이고 예측값이라고 부른다.

o1 = sigmoid( z3 ) = 0.60944600
o2 = sigmoid( z4 ) = 0.66384491

- 이제 할 일은 예측값과 실제값의 오차를 계산하기 위한 손실함수를 선택하는 것이다.
- 오차( Error )를 계산하기 위한 손실함수( Loss Function )로는 평균 제곱 오차( MSE )를 사용하는 것으로 가정한다.

\begin{equation*}
    E_{o1}=\frac{1}{2}(target_{o1}-output_{o1})^{2} = 0.02193381
\end{equation*}   
\begin{equation*}
    E_{o2}=\frac{1}{2}(target_{o2}-output_{o2})^{2} = 0.00203809
\end{equation*}   
\begin{equation*}
    E_{total} = E_{o1} + E_{o2} = 0.02397190
\end{equation*}

- 위식에서 실제값은 target, 순전파를 통해 나온 예측값은 output으로 표현
- 각 오차를 모두 더하면 \begin{equation*}E_{total}\end{equation*}가 된다.

- https://medium.com/@14prakash/back-propagation-is-very-simple-who-made-it-complicated-97b794c97e5c =>역전파에 대한 설명 잘해놓은 사이트

## 3. 역전파 1단계( Back Propagation Step 1 )

- 순전파가 입력층에서 출력층으로 향한다면 역전파는 반대로 출력층에서 입력층 방향으로 계산하면서 가중치를 업데이트해 간다.
- 출력층 바로 이전의 은닉층을 N층이라고 하였을 때, 출력층과 N층 사이의 가중치를 업데이트하는 단계를 역전파 1단계로 하고 과정을 알아본다.

![Alt text]( nn3_final.png )

- 역전파 1단계에서 업데이트 해야 할 가중치는 W5, W6, W7, W8 총 4개이다.
- 원리 자체는 동일하므로 W5에 대해서 업데이트 진행 과정을 이해해 본다.
- 경사 하강법을 수행하려면 가중치 W5를 업데이트 하기 위해서
\begin{equation*}\frac{\mathrm{d}E_{total}}{\mathrm{d}W_{5}} 를 계산해야 한다. \end{equation*}

\begin{equation*}
   \frac{\mathrm{d}E_{total}}{\mathrm{d}W_{5}} 를 계산하기 위한 미분의 연쇄법칙( Chain Rule )에 따라 다음과 같이 풀어 쓸 수 있다.
\end{equation*}

\begin{equation*}
   \frac{\mathrm{d}E_{total}}{\mathrm{d}W_{5}} = 
\frac{\mathrm{d}E_{total}}{\mathrm{d}o_{1}}\text{x}
\frac{\mathrm{d}o_{1}}{\mathrm{d}z_{3}}\text{x}\frac{\mathrm{d}z_{3}}{\mathrm{d}W_{5}}
\end{equation*}

- 위의 식에서 우변의 세 개의 각 항에 대해서 순서대로 계산한다.
- 첫번째 항에 대한 계산은 다음과 같다.

\begin{equation*}
    E_{total} = \frac{1}{2}(target_{o1}-output_{o1})^{2} + \frac{1}{2}
(target_{o2}-output_{o2})^{2}
\end{equation*}

\begin{equation*}
    \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{1}}는 다음과 같다.
\end{equation*}

\begin{equation*}
    \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{1}} = 2 \text{x} \frac{1}{2}( target_{o1} - output_{o1} )^{2-1} \text{X} (-1) + 0
\end{equation*}

\begin{equation*}
    \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{1}} = -( targaet_{o1}-output_{o1} ) = -( 0.4 - 0.60944600 ) = 0.20944600
\end{equation*}

- 두번째 항에서 o1값은 시그모이드 함수의 출력값이다.
- 시그모이드 함수의 미분은 f( x ) X ( 1 - f(x) )이다.
- 두번째 항의 미분 결과는 다음과 같다.

\begin{equation*}
    \frac{\mathrm{d}o_{1}}{\mathrm{d}z_{3}} = o_{1} \text{x}( 1 -o_{1} ) = 0.60944600 ( 1 - 0.60944600 ) = 0.23802157 
\end{equation*}

- 세번째 항의 h1의 값과 동일하다.

\begin{equation*}
    \frac{\mathrm{d}z_{3}}{\mathrm{d}W_{5}} = h_{1} = 0.51998934
\end{equation*}

- 우변의 모든 항을 계산한 결과를 곱하면

\begin{equation*}
    \frac{\mathrm{d}E_{total}}{\mathrm{d}W_{5}} = 0.20944600 \text{x} 0.23802157  \text{x} 0.51998934 = 0.2592286
\end{equation*}

- 경사 하강법을 통해 가중치를 업데이트 한다.
- 하이퍼파라미터인 학습률 a(alpha)는 0.5로 가정한다.

\begin{equation*}
    W_{5}^{+} = W_{5}-\alpha\frac{\mathrm{d}E_{total}}{\mathrm{d}W_{5}} = 0.45 - 0.5 \text{x} 0.02592286 = 0.43703857
\end{equation*}

\begin{equation*}
\frac{\mathrm{d}E_{total}}{\mathrm{d}W_{6}} = \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{1}} \text{×} \frac{\mathrm{d}o_{1}}{\mathrm{d}z_{3}} \text{×} \frac{\mathrm{d}z_{3}}{\mathrm{d}W_{6}} → W_{6}^{+}=0.38685205
\end{equation*}

\begin{equation*}
\frac{\mathrm{d}E_{total}}{\mathrm{d}W_{7}} = \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{2}} \text{×} \frac{\mathrm{d}o_{2}}{\mathrm{d}z_{4}} \text{×} \frac{\mathrm{d}z_{4}}{\mathrm{d}W_{7}} → W_{7}^{+}=0.69629578
\end{equation*}

\begin{equation*}
\frac{\mathrm{d}E_{total}}{\mathrm{d}W_{8}} = \frac{\mathrm{d}E_{total}}{\mathrm{d}o_{2}} \text{×} \frac{\mathrm{d}o_{2}}{\mathrm{d}z_{4}} \text{×} \frac{\mathrm{d}z_{4}}{\mathrm{d}W_{8}} → W_{8}^{+}=0.59624247
\end{equation*}

## 4. 역전파 2단계( Back Propagation Step 2 )

![Alt text]( nn4.png )

- 1단계를 완료한 후 입력층 방향으로 이동하며 역전파 1단계와 같은 방식으로 매개 변수를 변경하는 과정을 수행한다.

#### 인공 신경망의 학습은 오차를 최소화하는 가중치를 찾는 목적으로 순전파와 연전파를 반복하는 것을 말한다.